# Soccer Mobile
<!-- Introduction of the project -->

**Students: Chengkun Li, Jiangfan Li, Georgios Apostolides, Lucas Represa**

## I. Introduction

In this project the goal is to combine vision, path planning, local navigation, and filtering in order to make Thymio robot navigate trough a map towards a goal.  

As the constraints are established we were free to chose our own implementation. To begin with, a camera is used in order to provide the vision information. The vision module works along with the global path and filter modules extracting estimating and computing the necessary map information, including the robot pose, robot pose, map, static obstacles, and the goal position. 

Indeed, a Kalman filter performs the estimations of the robot pose. Afterwards, the A* algorithm computes the optimal path. Following this, a global controller gives instructions to the motors for them to follow the optimal path. Finally, a local navigation module is implemented in order to provide a vision-free information to Thymio for the most spontaneous events such as a dynamic obstacle.

## II. Implementation

**Import Libraries**

In [ ]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import numpy as np
import time
import random

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [20, 10]

**Thymio Connection**

In [ ]:
import tdmclient.notebook
await tdmclient.notebook.start()

**Environement**

In order to create our soccer field environnement we printed an A1 sheet with the corresponding field. Corners are represented by [...]. A camera performs the vision part by transferring frames to the vision module. 

## Geometry

In [ ]:
from geo import *

## Vision


In [ ]:
from vision import *
vp = VisionProcessor()
#vp.open()
#img = vp._getImage()
img = cv2.imread("img/example.jpg")
plt.imshow(cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

### Image Processing

#### Color Filtering

In [ ]:
color = 'green'
green_mask = VisionProcessor.color_filter(img, color=color)
plt.imshow(cv2.cvtColor(green_mask, cv2.COLOR_RGB2BGR))

#### Corner Detection

##### 1. Using Image Processing

In [ ]:
# 1. find the biggest green part

gray = cv2.cvtColor(green_mask, cv2.COLOR_BGR2GRAY)
(T, thresh) = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)
# morphology operation against noise
closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, np.ones((10, 10)))
# extract contours
_,contours,hierarchy = cv2.findContours(closing,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Taking the one with the largest area # serve as the opening operation to some extent
c = max(contours, key = cv2.contourArea)
# use a convexHull against noise
hull = cv2.convexHull(c)
hull_img = np.zeros_like(img)     
length = len(hull)
for i in range(len(hull)):
    cv2.line(hull_img, tuple(hull[i][0]), tuple(hull[(i+1)%length][0]), (255,0,0), 2)
hull_img = cv2.cvtColor(hull_img, cv2.COLOR_RGB2GRAY)
plt.imshow(hull_img)    
_,convexcontour,_ = cv2.findContours(hull_img,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
ch = img.copy()
cv2.drawContours(ch,convexcontour,-1,(255,0,0),1)
#plt.imshow(cv2.cvtColor(ch, cv2.COLOR_RGB2BGR))      

In [ ]:
# 2. get 4 corners
lines = VisionProcessor.divide4(np.array(hull))
ll = lines[np.argmax([abs(l[-1]) for l in lines])]
lines.remove(ll)
rl = lines[np.argmax([abs(l[-1]) for l in lines])]
lines.remove(rl)
ul = lines[0]
dl = lines[1]
corners = []
i = 1
ch = img.copy()
for h in [ul, dl]:
    for v in [ll, rl]:
        x, y = VisionProcessor.intersection(h[0], h[1], -h[2], v[0], v[1], -v[2])
        corners.append([x, y])
        print("corner",x, y)
        cv2.circle(ch, (x, y), 3, (0, 255, 0), -1)
        cv2.putText(ch,str(i), (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        i += 1
        
corners = np.array(corners)

plt.imshow(ch)


##### 2. Using aruco

In this part we get the pixel coordinates of all four corners by using the aruco markers.

In [ ]:
from vision import *
vision_processor = VisionProcessor()
img = cv2.imread("img/test.jpg")
plt.imshow(cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

In [ ]:
corners, a_image = vision_processor.visualize_aruco(img)
corners
plt.imshow(cv2.cvtColor(a_image, cv2.COLOR_RGB2BGR))

#### Perspective Projection
[TODO]

In [ ]:
M = VisionProcessor.align_field(corners)
wraped = VisionProcessor.warp(img, M)
plt.imshow(cv2.cvtColor(wraped, cv2.COLOR_RGB2BGR))


In [ ]:
# import numpy as np
# import cv2

# cap = cv2.VideoCapture(3)

# # fourcc = cv2.VideoWriter_fourcc(*'XVID')
# # out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640,480))

# while(cap.isOpened()):
#     ret, frame = cap.read()
#     if ret==True:
# #         frame = cv2.flip(frame,0)

#         image=resize_image(frame,80)
# #         image=align_field(image)
# #         out.write(frame)

#         cv2.imshow('frame',image)
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break
#     else:
#         break
# cap.release()

# # out.release()

# cv2.destroyAllWindows()

#### Box Detection

#### Ball Detection

### Detect Objects

#### The Field

#### The obstacles

In [ ]:
gMap = vp.getMap(wraped)

In [ ]:
print(gMap.scale)
print(Thymio_Size/gMap.scale)

#### The ball

#### Thymio

In [ ]:
thymip_state_camera = vp._getThymio()

## Global Navigation
This module aims to plan a path from the start to the goal.
The result will be represented in form of a list of Waypoints.

In [ ]:
### This is a fake map
# h, w = 40, 30
pStart = Pos(0,0)
pBall = Pos(gMap.height-1,gMap.width-1)
# rmap = GridMap(h, w, 0.01)
gMap.set_start(pStart)
# obslist = [Pos(random.randint(10,h-11),random.randint(10,w-11)) for _ in range(3)]
# rmap.set_obs(obslist)


# obsmap = [[int(val) for val in li] for li in rmap.obs_map]
# obsmap[pStart.x][pStart.y] = 0.3
# obsmap[pBall.x][pBall.y] = 0.7
###
obsmap = gMap.obs_map
plt.imshow(obsmap)

### Pre-Processing
#### Enlarge the obstacles
After getting the environment map, we need to enlarge the obstacles to make sure every point is safe for thymio.

In [ ]:
from global_navigation import *
planner =  PathPlanner(gMap, path_simplification=False)
plt.imshow(planner.obs) # has auto enlarged the obs when load the map

In [ ]:
gMap.check(Pos(600, 1000))

#### Caculate the actual position
The goal position represent where that the head of thmio will reach; We need to calculate the actual center position of thymio for the goal.

In [ ]:
pGoal1 = planner.approach(pBall)
planner.set_goal(pGoal1)
print("Goal State:",pGoal1)
import copy
obs = copy.deepcopy(planner.obs)
obs[pStart.x][pStart.y] = 0.3
obs[pBall.x][pBall.y] = 0.7
obs[pGoal1.pos.x][pGoal1.pos.y] = 0.5
plt.imshow(obs)

### Path Planning
We implemented two ways of path planning, namely, A* and RRT.
A* is an optimal path planning algorithm.
RRT is usually applied for high-dimension path planning. For our project, if we get a quite large grid map, the computational cost will be high, and RRT can boost up the speed. We should note that, it's not a optimal algorithms.

In [ ]:
# the result of A*
# points in the same direction have been deleted
planner.method = "A*"
planner.neighbor = 8
apath = planner.plan()
nobs = copy.deepcopy(obs)
for p in apath:
    nobs[p.x][p.y] = 0.9
plt.imshow(nobs)

In [ ]:
# the result of RRT
planner.method = "RRT"
rrtpath = planner.plan()
nobs = copy.deepcopy(obs)
for p in rrtpath:
    nobs[p.x][p.y] = 0.9
plt.imshow(nobs)

### Post-Processing
#### Collect waypoints in same direction
The waypoints in same direction just have the same effect for path tracking, so we try to elimate some redandent points.

In [ ]:
apath = planner.collect_wps(apath)
nobs = copy.deepcopy(obs)
for p in apath:
    nobs[p.x][p.y] = 0.9
plt.imshow(nobs)

#### Shrink the waypoints
Try to connect the grandparent to grandchild directly.

In [ ]:
apath = planner.path_simplification(apath)
nobs = copy.deepcopy(obs)
for p in apath:
    nobs[p.x][p.y] = 0.9
plt.imshow(nobs)

#### Assign Orientation


In [ ]:
Global_path = planner.assign_ori(apath)
for s in Global_path:
    print(s)

## Local Navigation


The local navigation module allows to take advantage of the proximity sensors located on the five front horizontal proximity sensors. The objective is to bypass the unknown local obstacle for further re-computing of the controller to correct Thymio's speed.

- **Input**

    - Horizontal proximity sensor values


- **Output**

    - Motion control command for robot translation and rotation

**Parameters**

| Name                | Meaning                                                      | Type (Unit) | Global |
| :------------------- | :------------------------------------------------------------ | :----------- | :------ |
| `max_speed`        |  Nominal speed                 | Int         |  |
| `obstThrL`      | Low obstacle threshold to switch state 1->0                | Int         |    |
| `obstThrH` | High obstacle threshold to switch state 0->1                    | Int         |   |
| `obstSpeedGain`         | Variation of speed according to the distance of obstacle | Int |   |
| `state`         | 0=global navigation, 1=local navigation | Bool |   |
| `obst`         | Measurements from left and right prox sensors | Int |   |



**Functions**

Two functions are basically doing the local avoidance. obs is meant to detect an obstacle and return the new state of the robot as stated in the parameters tabular. obstacle_avoidance updates the new speed to return to thymio. It will return the inputs to the motors function. 

In [ ]:
max_speed = 100       # nominal speed
speedGain = 2      # gain used with ground gradient
obstThrL = 10      # low obstacle threshold to switch state 1->0
obstThrH = 20      # high obstacle threshold to switch state 0->1
obstSpeedGain = 5  # /100 (actual gain: 5/100=0.05)

In [ ]:
@tdmclient.notebook.sync_var
def motors(left, right):
    global motor_left_target, motor_right_target
    motor_left_target = left
    motor_right_target = right
    
@tdmclient.notebook.sync_var   
def encoders():
    global motor_left_speed, motor_right_speed
    speed = []
    while len(speed) < 2 :
        speed = [motor_left_speed, motor_right_speed]
    return speed
    
@tdmclient.notebook.sync_var
def obstacle_avoidance():
    global prox_horizontal, state, obst, obstSpeedGain, speed0, speedGain 
    obst = [prox_horizontal[0], prox_horizontal[4]]
    
    state = 0
    if (obst[0] > obstThrH):
        state = 1
    elif (obst[1] > obstThrH):
        state = 1
        
    if state == 1:
        left_speed = max_speed // 2 + obstSpeedGain * (obst[0] // 100)
        right_speed = max_speed // 2 + obstSpeedGain * (obst[1] // 100)
        motors(left_speed, right_speed)
        return True
    else:
        return False

## Filtering
### Overview
The goal of the filtering module is to integrate (if available) the measurements (in our case, the vision output) with the information from the motor encoders to estimate a reasonable state for Thymio. To this end, we use Extended Kalman Filter (EKF) technique in our project. Compared to the regular Kalman Filter, Extended Kalman Filter is the nonlinear version of the Kalman filter which utilizes first (or second) order derivative to do the approximation.

## Robot States

The state of Thymio is represented as $$\mu_t = [x_t, y_t, \theta_t]^T$$ as shown in the figure below, corresponding to the x, y, and orientation of the robot as shown in the figure below.
<center><img src="Notebook_figures/robot_state.png" alt="" style="width: 800px;"/></center>
Where we denote the displacement in distance as D and displacement in orientation as T (for the sake of simplicity we omit the indices).

## State Space Representation of EKF
### Action and Measurement Model
The action and measurement model of EKF is shown below, where $f$ and $h$ are two nonlinear functions.
\begin{array}{l}\boldsymbol{x}_{t}=f\left(\boldsymbol{x}_{t-1}, \boldsymbol{u}_{t}\right)+\boldsymbol{w}_{t} \\\boldsymbol{z}_{t}=h\left(\boldsymbol{x}_{t}\right)+\boldsymbol{v}_{t}\end{array}

The prediction stage of EKF is described as:
$$\hat{\boldsymbol{x}}_{t \mid t-1}=f\left(\hat{\boldsymbol{x}}_{t-1 \mid t-1}, \boldsymbol{u}_{t}\right)$$


where the input of the system is a vector consisting of two encoder displacements of Thymio left/right wheels represented as $u_t = [\Delta S_r,\Delta S_l ]^T$
<center><img src="Notebook_figures/turning.png" alt="" style="width: 300px;"/></center>



In [ ]:
import filtering
# Motor Calibration

In [ ]:
pre_state = np.array([0.0, 0.0, 0]).reshape(-1, 1) # initial state
pre_cov = np.ones([3, 3]) * 0.03 # initial covariance
G_filter = filtering.KF(pre_state, pre_cov, qx=0.1, qy=0.1, qtheta=0.3, rl=0.1, rr=0.1, b=0.0927)
G_filter.timer = time.time()

In [ ]:
G_filter.states

## Motion Control

In [ ]:
pre_state = np.array([0.0, 0.0, 0]).reshape(-1, 1) # initial state
pre_cov = np.ones([3, 3]) * 0.03 # initial covariance
G_filter = filtering.KF(pre_state, pre_cov, qx=0.1, qy=0.1, qtheta=0.3, rl=0.1, rr=0.1, b=0.0927)
G_filter.timer = time.time()

In [ ]:
G_verbose = True
S_camera_interval = 1000 #ms
S_motion_interval = 200 #ms
S_track_interval = 0.2 #s

S_epsilon_dis = 0.005
S_epsilon_theta = 0.1

In [ ]:
class MotionController_JN:
    def __init__(self, time_interval = 10, # s 
                 eps_delta_r = 0.005, eps_delta_theta = 0.1,
                 max_speed = 100, 
                 speed_scale = 0.000315, # (m/s) / speed_in_motor_command; 0.000315 for speed<200; 0.0003 for speed \in (200,400)
                 rotate_scale = 0.05, # TODO (rad/s) / speed_in_motor_command
                 obstSpeedGain = 5,  # /100 (actual gain: 5/100=0.05)
                 verbose = False
                 ):
        """Motion Controller

        Connected with thymio interface
        Interface between high-level command and Thymio motion
        """
        self.interval = time_interval   # s, control frequency
        self.timer = time.time()
        self.displacement = [0, 0]
        self.speed = [0, 0]
        
        self.eps_delta_r = eps_delta_r
        self.eps_delta_theta = eps_delta_theta

        self.max_speed = max_speed
        self.speed_scale = speed_scale
        self.rotate_scale = rotate_scale
        self.obstSpeedGain = obstSpeedGain

        self.verbose = verbose

    # -- Local Navigation --
    def avoid(self):
        return obstacle_avoidance()

    # -- Path Tracking --        
    def path_tracking(self, waypoint, Thymio_state, theta_track = False):
        """Follow the path

        @return: waypoint reached
        """
        # 4 Track the waypoints
        # 4.1 Are we close enough to the next waypoint?  
        delta_r = Thymio_state.dis(waypoint)
        if delta_r < self.eps_delta_r:
            if self.verbose:
                print("Close to the point")
            # check the rotation
            delta_theta = Thymio_state.delta_theta(waypoint)
            if not theta_track or abs(delta_theta) < self.eps_delta_theta:
                if self.verbose:
                    print(Thymio_state,"Point Finished")
                return True
            else:
                self.rotate(delta_theta) #PULSE
        else:
            # 4.2 Go to the next waypoint
            headto_theta = Thymio_state.headto(waypoint)
            delta_theta = headto_theta - Thymio_state.ori
            delta_theta = Pos.projectin2pi(delta_theta)
            if self.verbose:
                print(F"headto_theta: {headto_theta}")
            if abs(delta_theta) > self.eps_delta_theta:#1.0:
                self.rotate(delta_theta)
            # elif abs(delta_theta) > self.eps_delta_theta:
            #     self.approach(delta_r, delta_theta)
            else:
                self.approach(delta_r, 0)
            return False

    # -- Movement --
    def approach(self, delta_r, delta_theta = 0):
        """approach to the goal point
        
            move with modification of direction
        """
        if self.verbose:
            print(F"approach to dr:{delta_r}, dt:{delta_theta}")
        # assume u only move <interval> s. 
        advance_speed = min(delta_r/self.interval/self.speed_scale * 20 + 20, self.max_speed)
        delta_speed = delta_theta/self.interval/self.rotate_scale
        if delta_speed > 0:
            delta_speed = min(delta_speed, self.max_speed/2)
            self.move(min(advance_speed, self.max_speed - 2*abs(delta_speed)), delta_speed)
        else:
            delta_speed = max(delta_speed, -self.max_speed/2)
            self.move(min(advance_speed, self.max_speed - 2*abs(delta_speed)), delta_speed)

    def rotate(self, delta_theta):
        """rotate in place
        """
        if self.verbose:
            print(F"rotate to dt:{delta_theta}")
        delta_speed = delta_theta/(self.interval)/self.rotate_scale
        if delta_speed > 0:
            self.move(0, min(delta_speed + 10, self.max_speed))
        else:
            self.move(0, max(delta_speed - 10, -self.max_speed))

    def move(self, vel, omega = 0):
        """
        move with transitional velocity and rotational velocity
        """
        if self.verbose:
            print(F"move with {vel}, {omega}")
        self._set_motor(vel - omega, vel + omega)

    def stop(self):
        """Stop both motors
        """
        self._set_motor(0, 0)

    def update_displacement(self):
        starter = time.time()
        interval = starter - self.timer
        self.timer = starter
        rls, rrs = encoders()
        # rls = int(rls * float(os.getenv("OFFSET_WHEELS")))
        #rls = rls if rls < 2 ** 15 else rls - 2 ** 16
        #rrs = rrs if rrs < 2 ** 15 else rrs - 2 ** 16
        rls = 0 if abs(rls) > self.max_speed * 1.1 else rls
        rrs = 0 if abs(rrs) > self.max_speed * 1.1 else rrs 
        self.displacement[0] += rls*interval*self.speed_scale
        self.displacement[1] += rrs*interval*self.speed_scale

    def _set_motor(self, ls, rs):
        ls = (int)(ls)
        rs = (int)(rs)
        # l_speed = int(ls / float(os.getenv("OFFSET_WHEELS")))
        #l_speed = ls if ls >= 0 else 2 ** 16 + ls
        #r_speed = rs if rs >= 0 else 2 ** 16 + rs           
        self.update_displacement()
        motors(ls, rs)

    def get_displacement(self):
        self.update_displacement()
        ret = self.displacement
        self.displacement = [0, 0]
        # if self.verbose:
        #     print(F"Displacement:{ret}")
        return ret

    
G_mc = MotionController_JN(verbose = G_verbose, time_interval = S_motion_interval/1000.0 )
print(S_motion_interval)
print(G_mc.interval)

In this main all the motion control and local avoidance functions are called in a while loop. This merges all the modules and communicates with the thymio thanks to the motion contol module.

In [ ]:
G_track_timer = time.time()
G_mc.get_displacement()

def localizate():
    """Track Where Thymio is"""
    global G_camera_timer
    starter = G_filter.timer
    # 3. Localization 
    # 3.1 odometer
    dsl, dsr = G_mc.get_displacement()
    # 3.2 With Vision
        # if starter - G_camera_timer > S_camera_interval:
        #     vision_thymio_state = G_vision._getThymio()
        #     # Vision Failed
        #     if vision_thymio_state is None:
        #         G_filter.kalman_filter(dsr, dsl)
        #     else:
        #         G_camera_timer = starter
        #         G_filter.kalman_filter(dsr, dsl, vision_thymio_state)
        # else:        
        #     G_filter.kalman_filter(dsr, dsl)
    G_filter.kalman_filter(dsr, dsl)
    G_filter.plot_gaussian()
    thymio_state = G_filter.get_state()
    return thymio_state

print(localizate())

In [ ]:
def reset():
    G_filter.states = [np.array([0.0, 0.0, 0]).reshape(-1, 1)] # initial state
    G_filter.covs = [np.ones([3, 3]) * 0.03]    
    G_track_timer = time.time()
    G_mc.get_displacement()
    print(localizate())
   
reset() 
## Fake Waypoints
Global_path = [State(Pos(0.1,0.0),0), State(Pos(0.1, 0.1),0.0),State(Pos(0.15, 0.15),0.0), State(Pos(0.2,0.2),0)]
Goal_state = Global_path[-1]

In [ ]:
debug_timer = 0.0
while True:
    starter = time.time()
    # 3. Localization
    Thymio_state = localizate()
    # 2.2.1 Finished?
    if Thymio_state.dis(Goal_state) < S_epsilon_dis \
        and abs(Thymio_state.ori - Goal_state.ori) < S_epsilon_theta:
        G_mc.stop()
        if G_verbose:
            print("Terminate Reached!")
        break
    # 2.2.2 Is there obstacles on the front?
    obs_front = G_mc.avoid() # do local navigation for, like, 10ms
    #     # TODO: replan
    if (not obs_front) and starter - G_track_timer > S_track_interval:
        # 4. Follow the path    # <-- The only task can run under low frequency
        reached = G_mc.path_tracking(Global_path[0], Thymio_state, len(Global_path) == 1)
        if reached:
            print(Global_path[0],"reached")
            Global_path = Global_path[1:]
            # assume Global_path is not empty because of 2.2.1
        G_track_timer = starter
    #loop_time = time.time() - starter
    time.sleep(0.15)
    if starter - debug_timer > S_track_interval:
        debug_timer = starter
        print(F"thymio: {Thymio_state}")

In [ ]:
motors(0,0)